In [1]:
import os
import glob
import warnings
warnings.filterwarnings("ignore")

from monai.transforms import LoadImage
import nibabel as nib
import numpy as np
from pydicom import dcmread
import pandas as pd
import tqdm
import rt_utils
import pydicom
from os import listdir
import tqdm
import json 
from datetime import datetime
import matplotlib.pyplot as plt

import SimpleITK as sitk
import sys
import os

In [2]:
!ls
meta_data_dir = 'BaseDonnée220Patients/'

 ApplicationUNETR.ipynb			     notebooks
 ArtifactsNonNormaliser.zip		     requirements.txt
 ArtifactsNormalisation.zip		     result
 ArtifactsRunAll4			     Run4Model.ckpt
 ArtifactsRuntest3			     RunAll
 assets					     RunAll1
 BaseDonnée220Patients			     RunAll2
 baseDonneeTest				     RunAll3NonNormaliser
'C79.3 2019-Mars2023_dataBasePatient.ods'    RunAll3Normaliser
'C79.3 2019-Mars2023_dataBasePatient.xlsx'   RunAll4
 colab-requirements.txt			     RunAll4.zip
 config2.yaml				     RunTest0
 config.yaml				     RunTest1
 converteurAllFileinPatientDir.ipynb	     RunTest10
 converteur.ipynb			     RunTest11
'Correction à faire'			     RunTest12
 dataset_0.json				     RunTest14
 DiceSpecSens_TrainValTest.ods		     RunTest16
 dicomTest				     RunTest17
 fichierNomsMetas.txt			     RunTest2
'LancerIAOrdi (copie).ipynb'		     RunTest3
 LancerIAOrdi.ipynb			     RunTest4
 meta					     RunTest5
 metastase_IA.zip			     RunTest6
 nifti_dataset				     RunTest7
 nifti_datasetNormalise

In [3]:
def miseEnDossier(patient_id):
    path_newDirRM = os.path.join(meta_data_dir, patient_id, "RM/")
    path_newDirMETA = os.path.join(meta_data_dir, patient_id, "META/")
    if not os.path.exists(path_newDirRM):
        os.mkdir(path_newDirRM)
    if not os.path.exists(path_newDirMETA):
        os.mkdir(path_newDirMETA)
    for mr in glob.glob(os.path.join(meta_data_dir, patient_id, "MR*")):
        path_dir, name_file= os.path.split(mr)
        os.rename(mr, str(path_newDirRM)+str(name_file))
    for rs in glob.glob(os.path.join(meta_data_dir, patient_id, "RS*")):
        path_dir, name_file= os.path.split(rs)
        os.rename(rs, str(path_newDirMETA)+str(name_file))

In [4]:
def save_image_as_nifti(array: np.ndarray, file_output: str):
    if len(array.shape) == 4:
        array = array[0]
    
    image = nib.Nifti1Image(array, np.eye(4))
    nib.save(image, file_output)

def fusionMask(mask):
        if (type(mask)==list):
            premMask = mask[0]
            for autMask in mask[1:]:
                premMask = np.where(autMask > 0, autMask, premMask)
            return premMask
        else:
            return mask

#tester mais sans résultats performant
"""def normalizationAllImageIRM(inputImage, arguments=1):
    print("debut normalization")
    inputImage = sitk.GetImageFromArray(inputImage)
    inputImage.SetOrigin((0,0,0))
    
    image = inputImage
    print("création masque")
    maskImage = sitk.OtsuThreshold(inputImage, 0, 1, 200)
    shrinkFactor = 1
    print("création correcteur")
    corrector = sitk.N4BiasFieldCorrectionImageFilter()
    numberFittingLevels = 4
    print("application correction")
    corrected_image = corrector.Execute(image, maskImage)
    #log_bias_field = corrector.GetLogBiasFieldAsImage(inputImage)
    #corrected_image_full_resolution = inputImage / sitk.Exp(log_bias_field)
    print("retransformation irm")
    corrected_image = sitk.GetArrayFromImage(corrected_image)

    return_images = {"input_image": inputImage,
                     "mask_image": maskImage,
                     "corrected_image": corrected_image}
    return return_images["corrected_image"]"""
    
listeName=[]
def load_dicom_with_mask(slices_folder, mask_file):
    rt_struct = rt_utils.RTStructBuilder.create_from(slices_folder, mask_file)
    maskAllAlone=[]
    for i in rt_struct.get_roi_names():
        if "GTV" in i or "recid" in i or "FRONTAL" in i or "GTC" in i or "Cerebelleux" in i or "PARIET" in i or "gtv" in i or "M1." in i or "M2." in i or "FRONTAL" in i:
            if "patient" not in i and "POST OP" not in i and "Préop" not in i and "External" not in i and "cav" not in i and "Cav" not in i and "PTV" not in i and "ANCIEN" not in i and "Ancien" not in i and "Anecien" not in i and "PTC" not in i:
                maskAllAlone.append(rt_struct.get_roi_mask_by_name(i).astype(np.float32))
                listeName.append(i)
    mask = fusionMask(maskAllAlone)
    img = np.array(LoadImage( image_only=True)(slices_folder))
    img = np.rot90(img)
    img = np.flip(img, 0)
    img = (img/np.max(img))*255
    #img=((img - np.min(img)) / (np.max(img) - np.min(img)))
    #img = normalizationAllImageIRM(img, 1)
    return img, np.array(mask)


def load_dicom_series_without_mask(patient_folder):
    img = LoadImage( image_only=True)(patient_folder)
    mask = np.zeros_like(img)
    #img = normalizationAllImageIRM(img, 1)
    return np.array(img), mask
  
def filter_patients_with_meta(patient_id: str) -> bool:
    miseEnDossier(patient_id)
    # if no folder in patient directory
    if len(glob.glob(os.path.join(meta_data_dir, patient_id, "*"))) == 0:
        return False

    # if no named folders "META" or "RM"
    if len(glob.glob(os.path.join(meta_data_dir, patient_id, "META"))) == 0 or len(glob.glob(os.path.join(meta_data_dir, patient_id, "RM"))) == 0:
        return False
    
    # if no dicom image in the META folder
    if len(glob.glob(os.path.join(meta_data_dir, patient_id, "META", "*"))) == 0:
        return False
    
    return True

def filter_patients_without_meta(patient_id: str) -> bool:
    miseEnDossier(patient_id)
    # if no folder in patient directory
    if len(glob.glob(os.path.join(meta_data_dir, patient_id, "*"))) == 0:
        return False

    # if no named folders "META" or "RM"
    if len(glob.glob(os.path.join(meta_data_dir, patient_id, "RM"))) == 0:
        return False

    if len(glob.glob(os.path.join(meta_data_dir, patient_id, "META"))) != 0:
        if len(glob.glob(os.path.join(meta_data_dir, patient_id, "META", "*"))) != 0:
            return False
    return True

def creaJson(emplacement, split=[7, 1, 2]):
    train=int(split[0]*len(patient_ids_replace)/10)
    val=int(split[1]*len(patient_ids_replace)/10)
    dataset = {
        "name": "Meta dataset", 
        "created": str(datetime.now()), 
        "reader": "nifti", 
        "dataset": 
            {"training": (patient_ids_replace[:train]), 
             "validation": (patient_ids_replace[train:(train+val)]), 
             "testing": (patient_ids_replace[(train+val):])
            }
    }
    with open(emplacement + 'dataset_0.json', 'w') as jsonFile:
        json.dump(dataset, jsonFile)


In [5]:
!ls DonnéesDicom/testPatient

ls: impossible d'accéder à 'DonnéesDicom/testPatient': Aucun fichier ou dossier de ce type


In [6]:
saving_dir = os.path.join(os.path.abspath(""), "nifti_dataset/")

print(saving_dir)

/home/aurelien/Documents/Segmentation Métastases cérébrales et méningiomes par IA/UNETR/metastase_IA/nifti_dataset/


In [10]:
#
patient_ids = [path.split(os.sep)[-1] for path in glob.glob(os.path.join(meta_data_dir, "*"))]
patient_id_with_meta = list(filter(filter_patients_with_meta, patient_ids))
patient_id_without_meta = list(filter(filter_patients_without_meta, patient_ids))

patient_ids_replace = [path.split(os.sep)[-1].replace(" ", "") for path in glob.glob(os.path.join(meta_data_dir, "*"))]
patient_id_with_meta_replace = []
patient_id_without_meta_replace = []
for i in patient_id_with_meta:
    patient_id_with_meta_replace.append(i.replace(" ", ""))

for i in patient_id_without_meta_replace:
    patient_id_without_meta_replace.append(i.replace(" ", ""))

print(patient_ids,patient_id_with_meta,patient_id_without_meta )

for patient_id in tqdm.tqdm(patient_id_without_meta):
    initial_path = os.path.join(meta_data_dir, patient_id)
    patient_id = patient_id.replace(" ", "")
    try:
        img, mask = load_dicom_series_without_mask(glob.glob(os.path.join(initial_path, "RM"))[0])
        os.makedirs(saving_dir+str(patient_id), exist_ok=True)
        
        save_image_as_nifti(img, os.path.join(saving_dir+str(patient_id), "image.nii.gz"))
      #os.makedirs(saving_dir+str(patient_id), exist_ok=True)
        save_image_as_nifti(mask, os.path.join(saving_dir+str(patient_id), "mask_meta.nii.gz"))
    except Exception as e:
        print("Error with", patient_id)
        print(e)

for patient_id in tqdm.tqdm(patient_id_with_meta):
    initial_path = os.path.join(meta_data_dir, patient_id)
    patient_id = patient_id.replace(" ", "")
    try:
        img, mask = load_dicom_with_mask(
            glob.glob(os.path.join(initial_path, "RM"))[0],
            glob.glob(os.path.join(initial_path, "META", "*.dcm"))[0],
        )
        os.makedirs(saving_dir+str(patient_id), exist_ok=True)
        save_image_as_nifti(img, os.path.join(saving_dir+(patient_id), "image.nii.gz"))
        os.makedirs(saving_dir+str(patient_id), exist_ok=True)
        save_image_as_nifti(mask, os.path.join(saving_dir+(patient_id), "mask_meta.nii.gz"))
    except Exception as e:
        print("Error with", patient_id)
        print(e)

df = pd.DataFrame([*patient_id_with_meta, *patient_id_without_meta], columns=["id"])
# df["has_meta"] = 0
df["has_meta"] = df["id"].isin(patient_id_with_meta)
df = df.sort_values("id")
df = df.reset_index(drop=True)
df.to_csv(os.path.join(saving_dir, "patients.csv"), index=False)
creaJson(saving_dir,split=[7, 1, 2])

print(listeName)

['201905984', '202209914', '202106842', '202006980', '202006391', '202000265', '202001838', '202301719', '201805382', '201900570', '201905202', '202006860', '202109581', '202003170', '202104352', '202103567', '202109494', '201709600', '202101477', '202205049', '201908838', '202007813', '201800255', '201910720', '202209468', '201904653', '202100606', '202111404', '201807688', '202007664', '202109865', '202000317', '202005507', '202211566', '201903761', '202101257', '202211870', '201906247', '202103577', '202210668', '202107795', '202101156', '201811269', '202211115', '202009111', '201907330', '202101013', '201908578', '202007818', '202110980', '201802539', '202204298', '202003115', '202107026', '202201119', '202000434', '202008248', '201903909', '201803717', '201902137', '201910386', '202201729', '202108526', '201902626', '202107635', '201709668', '202201953', '202109096', '201909977', '202007788', '201801714', '202000058', '201709127', '201906013', '202007259', '202008519', '202007567'

0it [00:00, ?it/s]
  0%|          | 1/215 [00:04<14:39,  4.11s/it]WARNING: In /work/ITK-source/ITK/Modules/IO/ImageBase/include/itkImageSeriesReader.hxx, line 477
ImageSeriesReader (0x2aad1e80): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.000199034

  1%|          | 2/215 [00:08<15:21,  4.33s/it]WARNING: In /work/ITK-source/ITK/Modules/IO/ImageBase/include/itkImageSeriesReader.hxx, line 477
ImageSeriesReader (0x27dc0710): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.0005

  3%|▎         | 7/215 [00:30<15:30,  4.48s/it]WARNING: In /work/ITK-source/ITK/Modules/IO/ImageBase/include/itkImageSeriesReader.hxx, line 477
ImageSeriesReader (0x1bbf9430): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.000298551

  4%|▍         | 9/215 [00:39<14:44,  4.29s/it]WARNING: In /work/ITK-source/ITK/Modules/IO/ImageBase/include/itkImageSeriesReader.hxx, line 477
ImageSeriesReader (0x1bbf9430): Non uniform sampling or mis

 43%|████▎     | 92/215 [07:57<12:02,  5.87s/it]WARNING: In /work/ITK-source/ITK/Modules/IO/ImageBase/include/itkImageSeriesReader.hxx, line 477
ImageSeriesReader (0x1bef2030): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.000199034

 46%|████▌     | 98/215 [08:30<11:16,  5.78s/it]WARNING: In /work/ITK-source/ITK/Modules/IO/ImageBase/include/itkImageSeriesReader.hxx, line 477
ImageSeriesReader (0x200f14b0): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.000927968

 46%|████▌     | 99/215 [08:35<10:57,  5.67s/it]WARNING: In /work/ITK-source/ITK/Modules/IO/ImageBase/include/itkImageSeriesReader.hxx, line 477
ImageSeriesReader (0x200f14b0): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.000521014

 47%|████▋     | 102/215 [08:53<10:53,  5.79s/it]WARNING: In /work/ITK-source/ITK/Modules/IO/ImageBase/include/itkImageSeriesReader.hxx, line 477
ImageSeriesReader (0xc595eb0): Non uniform sampling or missing slice

 94%|█████████▍| 203/215 [16:48<00:51,  4.26s/it]WARNING: In /work/ITK-source/ITK/Modules/IO/ImageBase/include/itkImageSeriesReader.hxx, line 477
ImageSeriesReader (0x12d5e7e0): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.000398068

 95%|█████████▌| 205/215 [16:58<00:46,  4.63s/it]WARNING: In /work/ITK-source/ITK/Modules/IO/ImageBase/include/itkImageSeriesReader.hxx, line 477
ImageSeriesReader (0x2297fb30): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.000398068

 97%|█████████▋| 208/215 [17:11<00:30,  4.38s/it]WARNING: In /work/ITK-source/ITK/Modules/IO/ImageBase/include/itkImageSeriesReader.hxx, line 477
ImageSeriesReader (0x1321c500): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.000398068

 97%|█████████▋| 209/215 [17:15<00:25,  4.29s/it]WARNING: In /work/ITK-source/ITK/Modules/IO/ImageBase/include/itkImageSeriesReader.hxx, line 477
ImageSeriesReader (0xb219590): Non uniform sampling or missing sl

Error with 202002528
list index out of range


100%|██████████| 215/215 [17:35<00:00,  4.91s/it]

['GTV ParietPost G 30Gy3F', 'GTV FRONT G', 'GTV FONT D', 'GTV TEMP D', 'FRONTAL G2', 'GTV TEMP G', 'M1.1_GTV_thalamus_D', 'GTV40Gy10Fr', 'GTVoccipiG', 'GTV', 'GTV2', 'GTV3', 'GTV4', 'GTVoccipitG', 'GTVparietpostDt', 'GTVparietpostG', 'GTV frontalG sup', 'GTVcerebeleuxG', '02/21 GTV Occipital Gh 36Gy6f', '02/21 GTV Occipital Dt 36Gy6f', '02/21 GTV Frontal Dt 36Gy6f', '02/21 GTV Temporal Dt 36Gy6f', 'P51 GTV-CapsuleIntD', 'P51 GTV CerebG 2', 'P51 GTV VERMIENNE G2', 'GTV CerebelleuseG', 'P11-gtvlitopfrontalG', 'P13-gtvcerebG', 'P31-gtvfrontDlat', 'GTV cerebeleux D', 'GTV occip G', 'GTV parietal D', 'GTV', 'M2.1_GTV_cereb_G', 'M2.2_GTV_temp_G', 'M2.3_GTV_front_G', 'M2.4_GTV_cereb_D', 'GTV', '1_GTV_cerebeleux_G_inf', '2_GTV_cerebeleux_G_sup_int_post', '3_GTV _frontal_G_sup_int_post', '4_GTV_frontal dt_sup_ext_post_ext', '5_GTV_parietal_D_ext_ant_sup', '6_GTV_parietal_D_int_post_inf', '7_GTV_occipital_D_int_sup_ant', '8_GTV_occipital_D_ext_ant_sup', '9_GTV_temporal_D_post_ext_sup', '10_GTV_t

In [8]:
#traduction des noms de métastases
import unidecode
def metaTraduction(listeNom):
    liste = []
    nonClasse=[]
    zonesCerebrales = ["Frontal", "Parietal", "Temporal", "Occipital", "Tronc", "Cervelet", "Diencephal"]
    for i in listeNom:
        val = unidecode.unidecode(i.lower())
        ajoutListe = ""
        if "fr" in val or "font" in val:
            ajoutListe += "Frontal"
        if "pari" in val :
            ajoutListe += "Parietal"
        if "temp" in val:
            ajoutListe += "Temporal"
        if "occ" in val:
            ajoutListe += "Occipital"
        if "tron" in val:
            ajoutListe += "Tronc"
        if "cer" in val or "crb" in val:
            ajoutListe += "Cervelet"
            
        if ajoutListe == "":
            ajoutListe = val
            nonClasse.append(ajoutListe)
        liste.append(ajoutListe)
    return liste, nonClasse
    
listeResult, nonClasse = metaTraduction(listeName)
print("il reste ", len(nonClasse), "noms de métastases mal classé parmis les ", len(listeResult)+len(nonClasse))
print(nonClasse)
print(listeResult)

il reste  160 noms de métastases mal classé parmis les  768
['m1.1_gtv_thalamus_d', 'gtv', 'gtv2', 'gtv3', 'gtv4', 'p51 gtv-capsuleintd', 'p51 gtv vermienne g2', 'gtv', 'gtv', '10_gtv_thalamus_d', 'gtv_l1_vermis_sup', 'gtv1', 'gtv2', 'gtv3', 'gtv4', 'gtv5', 'gtv6', 'gtv7', 'gtv8', 'gtv9', 'gtv10', 'gtv11', 'gtv', 'gtv', 'gtv', 'gtv', 'gtv', 'gtv', 'gtv periventricule d', 'm1.2_gtv_noyaucaudeg_inf-ant-int', 'gtv paietal g', 'p27 gtv gyrus d', '11_gtv_thalamus_g_ant-inf-ext', 'gtvvertexg', 'gtvpretki01.19', 'gtv 1', 'gtv 2', 'gtv3', 'gtv capsule g 36gy6f', 'gtv noyeau caude g', 'gtvinsulag', 'gtv1', 'gtv 1', 'gtv 2', 'gtv rolandique g', 'gtv periventriculaire g', 'gtv postcentg 36gy6f', 'gtv', 'gtv', 'gtv', 'gtv carrefourg', 'gtvsoust', 'gtvsustent2', 'gtvsustent1', 'gtv', 'gtv', 'gtv', 'gtv', 'gtvrecidive', 'encephale-gtv', 'gtv insul dt', 'gtv cav', 'gtv 11 para ventricule g', 'gtv', 'gtv1', 'gtv2', 'gtv4', 'gtv thalamo pedonc g', 'm2.10_gtv_thalamicd_int-inf-ant', 'gtv', 'gtv', 'gtv',